* Author: Gordon Erlebacher
* Date: 2022-03-03
* Purpose: convert Sent field to either date (nb seconds from a fixed time) or an empty field.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import pandas as pd
import numpy as np
import regex as rex
import pandas as pd
from datetime import datetime
from dateutil import parser
from dateutil.tz import gettz
from unidecode import unidecode

In [15]:
filenm = "output_0211.csv"
df = pd.read_csv(filenm)

In [17]:
def check_multiple_names(split_comma, el, new_to_list):
    flag = False
    for com in split_comma:
        # Why isn't \s included in ^\w
        # space(s), name, space(s), name, characters, email
        if rex.match(r'\s*(\w+)\s+(\w+)[^@]+@?', com):
            flag = True
            # remove everything at the end of the first email in com
        else:
            flag = False
            break
    if flag == True:   # Lines composed just of two names separated by spaces
        # print("el: ", el)
        new_to_list[-1] = rex.sub(',', ';', el)
        #print(f"__{new_to_list[-1]}")
    return flag

In [103]:
def check_email_or_two_words(split_comma, el, new_to_list):
    # email or two words in any order
    if split_comma[-1] == ' ' or split_comma[-1] == '':
        split_comma = split_comma[:-1]
    flag = False
    assign = 0
    mil_flag = False

    for com in split_comma:
        if rex.match(r'\s*([\w\-]+)\s+([\w\-]+)(?=[^@]*$)\W*$', com):
            assign += 1
        elif rex.match(r'\s*[\w\.]+@[\w\.]+\W*$', com):
            assign += 1
    if assign == len(split_comma):
        new_to_list[-1] = rex.sub(',', ';', el)
        return True
    else:
        return False

In [104]:
name = "  Nancy Miller "
rex.match(r'\s*(\w+)\s+(\w+)(?=[^@]*$)\W*$', name)

<regex.Match object; span=(0, 15), match='  Nancy Miller '>

In [105]:
def check_list_single_names(split_comma, el, new_to_list):   # REMOVE CELL WHEN DEBUGGED
    # Is split_comma a list of single names? 
    flag = False
    if split_comma[-1] == '' or split_comma[-1] == ' ':
        split_comma = split_comma[:-1]
    for com in split_comma:
        # Why isn't \s included in ^\w
        # space(s), name, space(s), name, characters, email
        if rex.match(r'\s*(\w+)\W*$', com):
            flag = True
            # remove everything at the end of the first email in com
        else:
            flag = False
            break
    if flag == True:   # Lines composed just of two names separated by spaces   (FLAG IS NOT TRUE!!!)
        # print("el: ", el)
        lst = []
        for i in range(0, len(split_comma), 2):
            try:
                lst.append(split_comma[i+1] + " " + split_comma[i])
            except:
                lst.append(split_comma[i])
        # print("lst: ", lst)
        #print(f"___{new_to_list[-1]}___")
        new_to_list[-1] = "; ".join(lst)
        #print(f"__{new_to_list[-1]}")
    return flag

In [106]:
def mixture_single_and_double_names(split_comma, el, new_to_list):
    """
    # Elements are either one word or two words, ordered arbitrarily
    
    Not processed correctly:
    ------------------------
    __el:  Gary Yordon, Robert Bass, Cathy Dunklin, John Kraynak, Brian Lawrence, BobSellers, James Allen
    __el: Van Riper, Jim, Hall, Wesley, Donahoe, Judy, Tedder, Wayne
    __el: ERROR... el:   Spellman, Hella R, Gale Workman



    """
    if split_comma[-1] == ' ' or split_comma[-1] == '':
        split_comma = split_comma[:-1]
    flag = False
    assign = 0
    nb_names = []

    for com in split_comma:
        # (?=[^@]*$) means to check that there are no @ to the right
        com = rex.sub(r'AICP', '', com)
        com = rex.sub(r'KristenBurke', 'Kristen Burke', com)  # Explode
        com = rex.sub(r'WayneTedder', 'Wayne Tedder', com)  # Explode
        com = rex.sub(r'Van Riper', 'VanRiper', com)  # Explode
        com = rex.sub(r'Hella R', 'HellaR', com)  # Explode
        if rex.match(r'\s*([\w\-]+)\s+([\w\-]+)\s*$', com):
            assign += 1
            nb_names.append(2)
        elif rex.match(r'\s*([\w\-]+)\s*$', com):
            assign += 1
            nb_names.append(1)
    if assign == len(split_comma):
        # if 1 name, combine the name and the following name
        # if 2 names, keep them together
        counter = 0
        string = ''
        i = 0
        # print(f"el: __{el}__")
        # print("nb_names: ", nb_names)
        # print("len(split_comma): ", len(split_comma))
        while True:
            # print("inside while, i= ", i,)
            if nb_names[i] == 2:
                string += split_comma[i] + ';'
                i += 1
                # print("if, nb_names: 2, new i: ", i, ", string: ", string)
            elif i < len(nb_names)-1 and nb_names[i] + nb_names[i+1] == 2:
                string += split_comma[i+1] + ' ' + split_comma[i] + ';' 
                i += 2
                # print("elif, nb_names: 1, 1, new i: ", i, ", string: ", string)
            else:
                string += split_comma[i] + ';'
                break
            if i >= len(nb_names): 
                break
                
#         print(f"__el: {el}")
        new_to_list[-1] = rex.sub(';$', '', string)
        if nb_names[-2] == 2 and nb_names[-1] == 1:
            print("ERROR?: el: ", el)
            print("new_to_list[-1] = ", new_to_list[-1])
            print()
        if rex.match(r'\s*\w+\s*$', new_to_list[-1]):   # <<< ERROR
            print("ERROR... el: ", el)  
            print("new_to_list[-1] = ", new_to_list[-1])
            print()
        # print(f"   __string: {string}")
        return True
    else:
        return False

In [202]:
name = ' Scof F. (satwell@admin.fsu.edu)'
com = rex.sub(r'\W*(\w+)\s+([A-H])\.', r'\1\2 ', name) 
print("com: ", com)

com = rex.sub(r'\W*\(([^\(]*)\)', r' \1', com)
print("com: ", com)

com = 'Mia Shargel (mshargel@gmail.com)'
com = rex.sub(r'\W*\(([^\(\)]*)\)', r' \1', com)
print("com: ", com)

com = 'Brian T.'
com = rex.sub(r'\b(\w+)\s+([A-Z])\.', r'\1\2 ', com, flags=rex.U) 
print(com)

com = "  Scott Williams  scott@hunterandharp.com "
com = unidecode(com)
com   = rex.match(r'\W*([\w\-]+)\s+([\w\-]+)\s+\W*([\w\.\_]+@[\w\.\-]+)', com, flags=rex.U)
# com = rex.match(r'\W*([\w\-]+)\s+([\w\-]+)', com, flags=rex.U)
print(com)


com:  ScofF  (satwell@admin.fsu.edu)
com:  ScofF satwell@admin.fsu.edu
com:  Mia Shargel mshargel@gmail.com
BrianT 
<regex.Match object; span=(0, 41), match='  Scott Williams  scott@hunterandharp.com'>


In [374]:
def mixture_email_single_and_double_names(split_comma, el, new_to_list):
    """
    # Elements are either (one word or email) or two words, ordered arbitrarily
    
    Not processed correctly:
    ------------------------
    __el:  Gary Yordon, Robert Bass, Cathy Dunklin, John Kraynak, Brian Lawrence, BobSellers, James Allen
    __el: Van Riper, Jim, Hall, Wesley, Donahoe, Judy, Tedder, Wayne
    __el: ERROR... el:   Spellman, Hella R, Gale Workman
    ___el: ERROR... el:   Faris, Alison McCray, Patricia Edwards, Ashley, Bowens, Spencer, Barber, Cynthia, Goad, Reese, Lavin, Raoul, paige.tallahasseedowntown@gmail.com, Gaines, Jerome
    RROR?: el:   Faulkenberry, Gary Yordon, brian.cook@HCAHealthcare.com, dale Neeley, Jeneane Dunkle, Tim
    ERROR?: el:   andrew.gillum@talgov.com, rick.fernandez@talgov.com, scof, Nancy Miller, gil.ziﬀer@talgov.com,  (WHAT is "scof"?)
RROR?: el:   Adrianne Miller, Aubry Hawks, Anneliese  , Gary Yordon, Cynthia.Barber@talgov.com
ERROR... el:   Faulkenberry, Gary Yordon, brian.cook@HCAHealthcare.com, dale Neeley, Jeneane Dunkle, Tim
ERROR?: el:   jackpofer@me.com, dougwray@comcast.net, Lauren Wray, adam, Susan Moore, Judi Pofer, Chuck

 SBELLEME@broward.org, chris.marlin@lennar.com, SPa†son@rsilink.com, lauren@laurenbook.com, mmk@portbiz.org, 
 scernak@broward.org, Chrisene.Wydra@lennar.com, claire@laurenbook.com, afazzano@broward.edu, tbain@miamigardens-ﬂ.gov, 
 sdick@talchamber.com, Pingree, Ben, Kerri.Post@VisitTallahassee.com, Fernandez, Rick, Curry, Chris, Maddox, Scof, 
 paige.tallahasseedowntown@gmail.com, agonzalez@visi‰lorida.org, ron@sachsmedia.com, rgreen2505@me.com, 
 victoria@sachsmedia.com, lehirst@fsu.edu, scof@scofmaddox.com, Atwell, Scof F. (satwell@admin.fsu.edu), Bryant, 
 
 Scof F. (satwell@admin.fsu.edu)
 
 Aundra, Elyea, Janice, carmella@sachsmedia.com, Gregory Haile
 
 Wilcox, Barry, Wiebler, Brian T., Doherty, Megan, Bryant, Cherie
 
 Sampson-Jones, Lavonia <Lavonia.Sampson-Jones@talgov.com>, Johansen, Jay <Jay.Johansen@talgov.com
 
 Dr. Audra Pi_man, Straughn, Rachael, Paige Carter-Smith, Poole, Carrie
    """
    # print("enter mixture")
    # print("el: ", el)
    # print("split_comma: ", split_comma)
    if split_comma[-1] == ' ' or split_comma[-1] == '':
        split_comma = split_comma[:-1]
    flag = False
    assign = 0
    nb_names = []
    emails = []
    scotflag = False

    for com in split_comma:
        # print(f"==> com: __{com}__")
        # (?=[^@]*$) means to check that there are no @ to the right
        com = rex.sub(r'AICP', '', com)
        com = rex.sub(r'KristenBurke', 'Kristen Burke', com)  # Explode
        com = rex.sub(r'WayneTedder', 'Wayne Tedder', com)  # Explode
        com = rex.sub(r'VinceLong', 'Vince Long', com)  # Explode
        com = rex.sub(r'EdMurray', 'Ed Murray', com)  # Explode
        com = rex.sub(r'Alan"Rosenzweig', 'Alan Rosenzweig', com)  # Explode
        com = rex.sub(r'MaryBarley', 'Mary Barley', com)  # Explode
        com = rex.sub('MikeVasilinda', 'Mike Vasilinda', com)
        com = rex.sub(r'Van Riper', 'VanRiper', com)  # Explode
        com = rex.sub(r'Van Pelt', 'VanPelt', com)  # Explode
        com = rex.sub(r'Hella R', 'HellaR', com)  # Explode
        com = rex.sub(r'(II|III)\s+$', '', com)
        
        # if three strings, combine the first two (assumes first, middle and last name)
        # print("     ...bef: ", com)
        ## Cheap solution (need better one) to reduce 3 or 4 words down to 2 (no consideration of emails)
        com = rex.sub(r'(\w+)\s+(\w+)\s+(\w+)', r'\1\2 \3', com, flags=rex.U)
        com = rex.sub(r'(\w+)\s+(\w+)\s+(\w+)', r'\1\2 \3', com, flags=rex.U)
        com = rex.sub(r'(\w+)\s+(\w+)\s+(\w+)', r'\1\2 \3', com, flags=rex.U)
        # print("     ...aft: ", com)

        com = rex.sub(r'\b([\w\-]+)\s+([A-Z])\.', r'\1\2 ', com) 
        # Change (xxxx) to xxxxx where xxxx is anything without parenthetis
        com = rex.sub(r'\W*\(([^\(\)]*)\)', r' \1', com)
        
        com = rex.sub(r'(Dr\.)', r' ', com)
        # print(f"   end com: __{com}__")
 
        emails.append(False)
        email = None
        first_name = None
        last_name = None
        
        # 2 names
        if rex.match(r'\s*([\w\-]+)\s+([\w\-]+)\s*$', com):
            assign += 1
            nb_names.append(2)
        # 2 names and email
        elif rex.match(r'\W*([\w\-]+)\s+([\w\-]+)\s+\W*([\w\.\-]+@[\w\.\-]+)', com, flags=rex.U):
            match = rex.match(r'\W*([\w\-]+)\s+([\w\-]+)\s+\W*([\w\.\-]+@[\w\.\-]+)', com, flags=rex.U)
            assign += 1
            nb_names.append(2)
            emails[-1] = True
            first_name = match.group(1)
            last_name = match.group(2)
            email = match.group(3)
        # 1 name 
        elif rex.match(r'\s*([\w\-]+)\s*$', com):
            assign += 1
            nb_names.append(1)
        # single email
        elif rex.match(r'\W*[\w+\._]+@[\w+\.\-]+', com, flags=rex.U):
            assign += 1
            nb_names.append(1)
            emails[-1] = True
        # single name + email
        elif rex.match(r'\s*([\w\-]+)\s\W*([\w+\.\-]+@[\w\.\-]+)', com, flags=rex.U):
            # print("...match")
            match = rex.match(r'\s*([\w\-]+)\s\W*([\w\.\-]+@[\w\.\-]+)', com, flags=rex.U);
            # print("match: ", match)
            #print("... com: ", com, ",     match: ", match.groups())
            assign += 1
            nb_names.append(1)
            emails[-1] = True
            first_name = match.group(1)
            email = match.group(2)
            # print("name: ", name, ",   email: ", email)
        # else:
        #     print("NO MATCH, com: ", com)

    # print("assign:  ", assign, ",    len(split_comma)= ", len(split_comma))      
    if assign == len(split_comma):
        # if 1 name, combine the name and the following name
        # if 2 names, keep them together
        counter = 0
        string = ''
        i = 0
        # print(f"el: __{el}__")
        # print("nb_names: ", nb_names)
        # print("len(split_comma): ", len(split_comma))
        while True:
            # print("inside while, i= ", i,)
            if nb_names[i] == 2:
                string += split_comma[i] + ';'
                i += 1
                # print("if, nb_names: 2, new i: ", i, ", string: ", string)
            elif emails[i] == True and not first_name:
                string += split_comma[i] + ';'
                i += 1
            elif i < len(nb_names)-1 and nb_names[i] + nb_names[i+1] == 2:
                if first_name:
                    string += first_name + ' ' + split_comma[i] + ' ' + email + ';' 
                else:
                    string += split_comma[i+1] + ' ' + split_comma[i] + ';' 
                i += 2
                # print("elif, nb_names: 1, 1, new i: ", i, ", string: ", string)
            else:
                string += split_comma[i] + ';'
                break
            if i >= len(nb_names): 
                break
                
        # Remove last semi-colon if end of string
        new_to_list[-1] = rex.sub(';\s*$', '', string)
        # print("new_to_list[-1]= ", new_to_list[-1])
        # if nb_names[-2] == 2 and nb_names[-1] == 1:
        #     print("ERROR?: el: ", el)
        #     print("new_to_list[-1] = ", new_to_list[-1])
        #     print()
        # if rex.match(r'\s*\w+\s*$', new_to_list[-1]):   # <<< ERROR
        #     print("ERROR... el: ", el)  
        #     print("new_to_list[-1] = ", new_to_list[-1])
        #     print()
        # print(f"   __string: {string}")
        return True
    else:
        return False

In [369]:
for el in to_lst[0:7]:
    print(el)

Charla Lucas, Lee Daniel, john@moonevents.com, marycfrederick@aol.com, Charles Hargraves, Autumn Calder,Roxanne Manning, Charles Hargraves, rbedard@rrbsystems.com, Daniel Fendley, Dee Crumpler, stonearch99@yahoo.com,rlc2998@gmail.com, jrfsu@comcast.net, Louis Norvell, vyoung26@comcast.net, scottscarswell@gmail.com, Autumn Calder,dbastriff@gmail.com, Ashley Edwards, Derek Friend, Andrea Griffin  Neighborhood Services , khgrey@aol.com, Tomica Smith,Linda Hudson, Wayne Tedder, lawleyscott@gmail.com, Mario Palmentieri, Ryan Grindler,  piers.rawling@gmail.com , cora@scottcarswellpresents.com


In [370]:
name = " Dr. "
rex.sub(".*\bDr.", '', name)

name = "This island is beautiful"
rex.sub("\bis\b", "gordon", name)

'This island is beautiful'

In [377]:
to_lst = df['CC'].values

In [378]:
not_processed = []
new_to_list = []

# to_lst = [
# # #     'Scott Williams  scott@hunterandharp.com , cmusgrove@cfobeanteam.com  cmusgrove@cfobeanteam.com ,\
# # #     Breck Brannen  breck@penningtonlaw.com , Parker, Michael  Michael.Parker@talgov.com , Baker, \
# # #     Sherri  Sherri.Baker@talgov.com , Wild, Ann J.  Ann.Wild@talgov.com'
# #    # "Dr. Audra Pi_man, Straughn, Rachael, Paige Carter-Smith, Poole, Carrie"
# #     # 'Sampson-Jones, Lavonia  Lavonia.Sampson-Jones@talgov.com , Johansen, Jay  Jay.Johansen@talgov.com'
# # 'Charla Lucas, Lee Daniel, john@moonevents.com, marycfrederick@aol.com, Charles Hargraves, Autumn Calder,\
# # Roxanne Manning, Charles Hargraves, rbedard@rrbsystems.com, Daniel Fendley, Dee Crumpler, stonearch99@yahoo.com,\
# # rlc2998@gmail.com, jrfsu@comcast.net, Louis Norvell, vyoung26@comcast.net, scottscarswell@gmail.com, Autumn Calder,\
# # dbastriff@gmail.com, Ashley Edwards, Derek Friend, Andrea Griffin  Neighborhood Services , khgrey@aol.com, Tomica Smith,\
# # Linda Hudson, Wayne Tedder, lawleyscott@gmail.com, Mario Palmentieri, Ryan Grindler,  piers.rawling@gmail.com , cora@scottcarswellpresents.com'
# ]

for el1 in to_lst[0:]:
    # print("el1: ", el1)
    if type(el1) != str:
        #not_processed.append(el1)
        new_to_list.append('NOT VALID')
        continue
    el = unidecode(el1)
    el = rex.sub(r'[\'\"]', '', el)
    # transform abbreviations of the form A. by A
    #el = rex.sub(r' (\w)\.', r' \1 ', el)  # adding this increases non-processed to 436 from 397
    # Remove (\w+)
    #el = rex.sub(r'\([\w\-\s]+\)', ' ', el)
    el = rex.sub(r'[\{\}\(\)\<\>\[\]]', ' ', el)
    if rex.match(r'.* (that|the) ', el):
        new_to_list.append('NOT VALID')
        continue
    new_to_list.append(el)
    try:
        splits = el.strip().split(";")
    except:
        continue
    sz = len(splits)
    if sz == 1:
        split_comma = el.strip().split(',')
        if len(split_comma) <= 2:
            continue 
        split_at = el.split('@')
        if len(split_comma) == len(split_at) or len(split_comma) == (len(split_at) - 1):
            # simply replace commas by semi-colons
            continue
            
        # The com should be processed once before calling these different routines, with regex.sub 
        # TODO
        
        # each split by comma should be EITHER two words or have an email
        flag = check_multiple_names(split_comma, el, new_to_list)
        if flag: continue

        flag = check_list_single_names(split_comma, el, new_to_list)
        if flag: continue
        
        # process elements.
        # - If full name (2 words), assign 1. 
        # - if just an email (1 string), assign 1
        # if the string is composed of just 1's, replace commas by semi-colons
        
        flag = check_email_or_two_words(split_comma, el, new_to_list)
        if flag: continue
        
        # flag = mixture_single_and_double_names(split_comma, el, new_to_list)
        flag = mixture_email_single_and_double_names(split_comma, el, new_to_list)
        if flag: continue
        
        not_processed.append(el)

        # for com in split_comma:
        #     print(f"__{com}__")
        # print(len(split_comma), len(split_at), el)
        
        # I would also lines where each element is either: 
        #  two or more names separated by spaces followed by a string of characters including the @ with no spaces
        #   (which is more general)
        
print(len(not_processed), len(new_to_list), len(to_lst))


73 71143 71143


In [376]:
com = ' AndreaGriffin   Neighborhood Services  '
com = rex.sub(r'(\w+)\s+(\w+)\s+(\w+)', r'\1\2 \3', com)
print(com)

 AndreaGriffinNeighborhood Services  


In [379]:
for el in not_processed:
    print(f"==> {el}")

==>  Paige Carter-Smith, Parade Spring.me Tallahassee, Kring, Edward
==> Favors Thompson, Anita  Anita.Favors.Thompson@talgov.com , Dick, Cindy  Cynthia.Dick@talgov.com , Coe, Tom  Tom.Coe@talgov.com , Leadership Team  LTExchange@talgov.com , Mayor & City Commissioners  citycommis@talgov.com
==> Scott Williams  scott hunterandhar .com , cmusgrove@cfobeanteam.com Breck Brannen Parker, Michael Baker, Sherri Wild, Ann  Ann.Wild@talgov.com
==>  Gant, Cassandra, Davis, Tim, Kristal Middlebrook  Kristal.Middlebrook@goisc.com , ISS/TI Server
==>  Thursday, November 12, 2015 4:26 PM
==>  Palmer, Steven, Lee, Jimmy, Davenport, Byron, Smith, Clint, McGarrah, Rob, Goad, Reese, Tedder, Wayne, Drose, Tina, Tadros, Mike, Ameen, Sam, Mayfield, Stephen, Gomez, Blas, Hosey, Jus.n, Reinert, Richard, Napier, Tom, Airom, Ross, Harrison, Tyler, Fowler, Bill, Shafer, Steve, paige.tallahasseedowntown@gmail.com, Faris, Alison, JT BurneDe, Kim Rivers  kim@inkbridge.com , Rodney Lewis, Sean Marston, Edwards, As

In [67]:
name = '  @ asf'
rex.match(r'[^\w]+@', name)

<regex.Match object; span=(0, 3), match='  @'>

In [ ]:
name = '2/16/17 3:25 PM (GMT-05:00)'
name = rex.sub(r'(\d?\/\d)\/(\d\d )', r'\1/20\2', name)
rex.match(r'(\d+?\/\d+?)\/(\d\d )', name)

In [ ]:
exceptions = []
date_dict = []
time_zone = []
for sent in sent_lst:
    normalize(sent, exceptions, date_dict, time_zone)
    
len(exceptions), len(sent_lst), df.shape

In [ ]:
exceptions

* 'Friday, May 13, 201611:18 AM': datetime.datetime(2022, 5, 13, 18, 16, 11),
* 

In [ ]:
dates_orig = []
dates_new = []
dates_date = []
date_adj = []
timestamp = []
for i, (tz_el, el) in enumerate(zip(time_zone, date_dict)):
    dates_orig.append(el[0])
    dates_new.append(el[1])
    dates_date.append(el[2])
    try:
        # subtract 5 hours to convert back to Tallahassee time
 
        if False:  # take daylight savings properly into account
            if tz_el[1] == 'S':  # adjust time if Savings
                timestp -= 3600
        else:  
            timestp = el[2].timestamp() - 5 * 3600 
            
        timestamp.append(timestp)
        # Transform time from timestamp back to UTC
        dtime = datelib.timestampToDateTimeUTC(timestp)
        date_adj.append(dtime)
    except:
        timestamp.append(-1)
        date_adj.append(('',''))
    
df1 = pd.DataFrame({'orig':dates_orig, 'new':dates_new, 'date':dates_date, 'TZ': time_zone, 'date_adj': date_adj, 'timestamp': timestamp})
print(len(date_dict), len(dates_orig), df.shape)

df1.to_csv("dates.csv", index=0)

In [ ]:
df.shape, df1.shape

In [ ]:
# Add new columns: new date and time, and number of seconds since 1970
# timestamp: seconds since 1970
# dates_orig: original send column
# date_adj[0]: adjusted date
# date_adj[1]: adjusted time
adj_date = []
adj_time = []
for dat_tim in date_adj:
    adj_date.append(dat_tim[0])
    adj_time.append(dat_tim[1])
    
df['timestamp'] = timestamp
df['date_sent'] = adj_date
df['time_sent'] = adj_time
df.columns

df.to_csv("output_stats_timestamps.csv.gz", index=0)

In [ ]:
timestamp;
df.columns